# Higuchi Fractal Dimension

In [8]:
import numpy as np

# Load EEG data
EEG_data = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/eeg_data_with_channels.npy', allow_pickle=True)

# Adjust the list to match your data's channels
eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

def higuchi_fd(data, k_max):
    """Compute Higuchi Fractal Dimension of a time series.
    
    Parameters:
    data : list or np.array
        One-dimensional time series
    k_max : int
        Maximum delay (time offset)
        
    Returns:
    hfd : float
        Higuchi Fractal Dimension
    """
    N = len(data)
    L = []
    
    x = np.asarray(data)
    
    for k in range(1, k_max):
        Lk = []
        
        for m in range(0, k):
            Lkm = 0
            for i in range(1, int((N-m)/k)):
                Lkm += abs(x[m+i*k] - x[m+i*k-k])
            Lkm = Lkm*(N - 1)/(((N - m)/k)*k)
            Lk.append(Lkm)
            
        L.append(np.log(np.mean(Lk)))
    
    hfd = np.polyfit(np.log(range(1, k_max)), L, 1)[0]
    
    return hfd

hfd_values = []

for channel_data in EEG_data.T:  # .T because we need to iterate over channels
    hfd_channel = higuchi_fd(channel_data, k_max=10)  # Adjust k_max as needed
    hfd_values.append(hfd_channel)

print("Higuchi Fractal Dimensions for each channel:", hfd_values)

Higuchi Fractal Dimensions for each channel: [-0.36005497045124624, -0.23633437232263893, -0.37112149677594536, -0.08754714948970373, -0.11035468235736252, -0.13707288326129607, -0.3934048831939664, -0.24673709351805825, -0.0710953944331469, -0.10275985039837075, -0.3467667431744175, -0.24990709690042587, -0.21426902542688608, -0.09127876391896302, -0.08507749371463456, -0.25722852525504636, -0.2805514117387538, -0.2224151766390623, -0.2099877581545126, -0.0799982792792008, -0.20791211613648958, -0.33103053273893573, -0.2269624863539871, -0.2363840209397604, -0.2704812265220445, -0.31618623070687657, -0.2329294151814016, -0.20944694956243734, -0.2115303234095175, -0.19603356897036747, -0.20279111854623105, -0.2125008482366303]


# Save the results

In [9]:
# Save the results to a text file
results_file_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Fractal Analysis/Higuchi_Fractal_Dimensions.txt'
with open(results_file_path, 'w') as f:
    for channel, hfd in zip(eeg_channels, hfd_values):
        f.write(f"Channel {channel}: {hfd}\n")

print("Results saved to:", results_file_path)

Results saved to: /home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Fractal Analysis/Higuchi_Fractal_Dimensions.txt
